# Code that might be useful later

## Sklearn trick for clustering strings

In [2]:
class SklearnStringTrick:
    """
    String clustering trick shown at:
        https://scikit-learn.org/0.16/faq.html#how-do-i-deal-with-string-data-or-trees-graphs

    Class structure allows for method reuse over distinct data

    """

    def __init__(self, dataset=[]):
        self.dataset = dataset

    def lev_dist_sklearn_urls(self, x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        return levenshtein(self.dataset[i], self.dataset[j])

In [3]:
string_trick = SklearnStringTrick()

## Pattern for filtering IPv6 addresses

## My attempt for regex splitting domain and subdomains

## Alternative library for tld matching

#### N-grams

#### Non-functional version of TF-IDF

Crashes on the .toarray() function
Might be memory issue?

TfidfVectorizer works though

## TF-IDF using HashingVectorizer

Not working yet

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer


class IncrementalHashingVectorizer:
    def __init__(self, n_features=2**18, ngram_range=(1, 1), **kwargs):
        self.vectorizer = HashingVectorizer(
            n_features=n_features, ngram_range=ngram_range, **kwargs
        )
        self.partial_fit_calls = 0

    def partial_fit(self, X):
        if self.partial_fit_calls == 0:
            self.features_sum = np.zeros(self.vectorizer.n_features)
            self.feature_counts = np.zeros(self.vectorizer.n_features)

        transformed = self.vectorizer.transform(X)
        self.features_sum += transformed.sum(axis=0)
        self.feature_counts += transformed.getnnz(axis=0)
        self.partial_fit_calls += 1
        return self

    def transform(self, X):
        transformed = self.vectorizer.transform(X)
        idf_vector = np.log(self.partial_fit_calls / (1 + self.feature_counts)) + 1
        return transformed.multiply(idf_vector)


incremental_vectorizer = IncrementalHashingVectorizer(
    ngram_range=NGRAM_RANGE, analyzer="char"
)


incremental_vectorizer.partial_fit([doc])

transformed_documents = incremental_vectorizer.transform(documents)

## TF-IDF on batches

In [ ]:
# Create TfidfVectorizer instance for domain column
tfidf_domain = TfidfVectorizer(analyzer="char", ngram_range=NGRAM_RANGE)
# Fit and transform domain column
tfidf_domain_matrix = tfidf_domain.fit_transform(urls_df["domain"])

# Create TfidfVectorizer instance for subdomain column
tfidf_subdomain = TfidfVectorizer(analyzer="char", ngram_range=NGRAM_RANGE)
# Fit and transform subdomain column
tfidf_subdomain_matrix = tfidf_subdomain.fit_transform(urls_df["subdomain"])

# Create pandas dataframe from tfidf_matrix
tfidf_domain_df_tfidfv = pd.DataFrame(
    tfidf_domain_matrix.toarray(),
    columns=tfidf_domain.get_feature_names_out(),
    index=urls_df.index,
)
tfidf_subdomain_df_tfidfv = pd.DataFrame(
    tfidf_subdomain_matrix.toarray(),
    columns=tfidf_subdomain.get_feature_names_out(),
    index=urls_df.index,
)

# Concatenate the dataframes
# tfidf_df = pd.concat([tfidf_domain_df, tfidf_subdomain_df], axis=1)

In [ ]:
tfidf_domain_df_tfidfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638971 entries, 0 to 638970
Columns: 165105 entries, '01 to zzzi
dtypes: float64(165105)
memory usage: 786.0 GB


In [ ]:
tfidf_subdomain_df_tfidfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638971 entries, 0 to 638970
Columns: 162855 entries, '00 to zzzz
dtypes: float64(162855)
memory usage: 775.3 GB


## How to use LSA for TF-IDF

In [27]:
# Taken from https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#performing-dimensionality-reduction-using-lsa

lsa = make_pipeline(TruncatedSVD(n_components=10), Normalizer(copy=False))

X_domain_lsa = lsa.fit_transform(X_domain_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

Explained variance of the SVD step: 49.2%


### Attempts at parallelising the finding of optimal truncate ratio

In the end I decided to subsample

Might be useful in future development

The main issue: Threads copy _as I currently believe_ the whole dataset into their processing space. Since the matrix is large it takes a bunch of memory.

Possible solutions: 
- Start smaller number of workers
- Share data among workers (read-only)
    - Tried this, the sparse matrices cannot be shared _as I currently believe_
    
My solution:
- Subsample

In [25]:
from multiprocessing import Pool, shared_memory

# Convert the sparse matrices to COO format
X_domain_coo = X_domain_tfidf.tocoo()
X_subdomain_coo = X_subdomain_tfidf.tocoo()

# Create shared memory blocks for the COO data, row, and col attributes
X_domain_data_shm = shared_memory.SharedMemory(
    create=True, size=X_domain_coo.data.nbytes
)
X_domain_row_shm = shared_memory.SharedMemory(create=True, size=X_domain_coo.row.nbytes)
X_domain_col_shm = shared_memory.SharedMemory(create=True, size=X_domain_coo.col.nbytes)

X_subdomain_data_shm = shared_memory.SharedMemory(
    create=True, size=X_subdomain_coo.data.nbytes
)
X_subdomain_row_shm = shared_memory.SharedMemory(
    create=True, size=X_subdomain_coo.row.nbytes
)
X_subdomain_col_shm = shared_memory.SharedMemory(
    create=True, size=X_subdomain_coo.col.nbytes
)

# Copy the COO data, row, and col attributes to shared memory
X_domain_data_shared = np.ndarray(
    X_domain_coo.data.shape, dtype=X_domain_coo.data.dtype, buffer=X_domain_data_shm.buf
)
X_domain_row_shared = np.ndarray(
    X_domain_coo.row.shape, dtype=X_domain_coo.row.dtype, buffer=X_domain_row_shm.buf
)
X_domain_col_shared = np.ndarray(
    X_domain_coo.col.shape, dtype=X_domain_coo.col.dtype, buffer=X_domain_col_shm.buf
)

X_subdomain_data_shared = np.ndarray(
    X_subdomain_coo.data.shape,
    dtype=X_subdomain_coo.data.dtype,
    buffer=X_subdomain_data_shm.buf,
)
X_subdomain_row_shared = np.ndarray(
    X_subdomain_coo.row.shape,
    dtype=X_subdomain_coo.row.dtype,
    buffer=X_subdomain_row_shm.buf,
)
X_subdomain_col_shared = np.ndarray(
    X_subdomain_coo.col.shape,
    dtype=X_subdomain_coo.col.dtype,
    buffer=X_subdomain_col_shm.buf,
)

np.copyto(X_domain_data_shared, X_domain_coo.data)
np.copyto(X_domain_row_shared, X_domain_coo.row)
np.copyto(X_domain_col_shared, X_domain_coo.col)

np.copyto(X_subdomain_data_shared, X_subdomain_coo.data)
np.copyto(X_subdomain_row_shared, X_subdomain_coo.row)
np.copyto(X_subdomain_col_shared, X_subdomain_coo.col)

# Calculate the explained variance ratio for different numbers of components
max_n_components_domain = 600  # Adjust based on your needs
max_n_components_subdomain = 30  # Adjust based on your needs
start_n_components_domain = 500
start_n_components_subdomain = 10
step_domain = 100
step_subdomain = 10


def lsa_variance_ratio(args):
    (
        data_shm_name,
        row_shm_name,
        col_shm_name,
        shape,
        dtype,
        n_components,
        feature,
    ) = args

    data_shm = shared_memory.SharedMemory(name=data_shm_name)
    row_shm = shared_memory.SharedMemory(name=row_shm_name)
    col_shm = shared_memory.SharedMemory(name=col_shm_name)

    data = np.ndarray(shape[0], dtype=dtype[0], buffer=data_shm.buf)
    row = np.ndarray(shape[1], dtype=dtype[1], buffer=row_shm.buf)
    col = np.ndarray(shape[2], dtype=dtype[2], buffer=col_shm.buf)

    # Reconstruct the sparse matrix in CSR format
    X_tfidf_coo = sp.csr_matrix((data, (row, col)), shape=shape[3])
    start = time.time()
    lsa = make_pipeline(TruncatedSVD(n_components=n_components), Normalizer(copy=False))
    lsa.fit_transform(X_tfidf_coo)
    explained_variance_ratio = lsa[0].explained_variance_ratio_.sum()
    end = time.time()

    # Close the shared memory objects in the current process
    data_shm.close()
    row_shm.close()
    col_shm.close()

    return (n_components, explained_variance_ratio, end - start, feature)


# Parallelize the for loop using multiprocessing.Pool
with Pool() as pool:
    tasks_domain = [
        (
            X_domain_data_shm.name,
            X_domain_row_shm.name,
            X_domain_col_shm.name,
            (
                X_domain_coo.nnz,
                X_domain_coo.row.size,
                X_domain_coo.col.size,
                X_domain_tfidf.shape,
            ),
            (X_domain_coo.data.dtype, X_domain_coo.row.dtype, X_domain_coo.col.dtype),
            n_components,
            "domain",
        )
        for n_components in range(
            start_n_components_domain, max_n_components_domain + 1, step_domain
        )
    ]
    tasks_subdomain = [
        (
            X_subdomain_data_shm.name,
            X_subdomain_row_shm.name,
            X_subdomain_col_shm.name,
            (
                X_subdomain_coo.nnz,
                X_subdomain_coo.row.size,
                X_subdomain_coo.col.size,
                X_subdomain_tfidf.shape,
            ),
            (
                X_subdomain_coo.data.dtype,
                X_subdomain_coo.row.dtype,
                X_subdomain_coo.col.dtype,
            ),
            n_components,
            "subdomain",
        )
        for n_components in range(
            start_n_components_subdomain, max_n_components_subdomain + 1, step_subdomain
        )
    ]

    results_domain = pool.map(lsa_variance_ratio, tasks_domain)
    results_subdomain = pool.map(lsa_variance_ratio, tasks_subdomain)

times_explained_variance_domain = np.array(results_domain)
times_explained_variance_subdomain = np.array(results_subdomain)

# Open a file and use dump()
with open("domain_tev_final.pkl", "wb") as file:

    # A new file will be created
    pickle.dump(times_explained_variance_domain, file)


# Open a file and use dump()
with open("subdomain_tev_final.pkl", "wb") as file:

    # A new file will be created
    pickle.dump(times_explained_variance_subdomain, file)

# Clean up shared memory blocks
X_domain_data_shm.close()
X_domain_data_shm.unlink()
X_domain_row_shm.close()
X_domain_row_shm.unlink()
X_domain_col_shm.close()
X_domain_col_shm.unlink()

X_subdomain_data_shm.close()
X_subdomain_data_shm.unlink()
X_subdomain_row_shm.close()
X_subdomain_row_shm.unlink()
X_subdomain_col_shm.close()
X_subdomain_col_shm.unlink()

Process ForkPoolWorker-23:
Process ForkPoolWorker-13:
Process ForkPoolWorker-24:
Process ForkPoolWorker-29:
Process ForkPoolWorker-15:
Process ForkPoolWorker-19:
Process ForkPoolWorker-53:
Process ForkPoolWorker-14:
Process ForkPoolWorker-17:
Process ForkPoolWorker-44:
Process ForkPoolWorker-58:
Process ForkPoolWorker-57:
Process ForkPoolWorker-42:
Process ForkPoolWorker-51:
Process ForkPoolWorker-50:
Process ForkPoolWorker-54:
Process ForkPoolWorker-46:
Process ForkPoolWorker-45:
Process ForkPoolWorker-49:
Process ForkPoolWorker-52:
Process ForkPoolWorker-64:
Process ForkPoolWorker-56:

KeyboardInterrupt



In [ ]:
import time
from multiprocessing import Pool, shared_memory

import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# Dummy data, replace with your actual data
X_domain_tfidf = np.random.rand(10000, 2000)
X_subdomain_tfidf = np.random.rand(10000, 2000)

# Create shared memory blocks for the arrays
X_domain_shm = shared_memory.SharedMemory(create=True, size=X_domain_tfidf.nbytes)
X_subdomain_shm = shared_memory.SharedMemory(create=True, size=X_subdomain_tfidf.nbytes)

# Copy the arrays to shared memory
X_domain_tfidf_shared = np.ndarray(
    X_domain_tfidf.shape, dtype=X_domain_tfidf.dtype, buffer=X_domain_shm.buf
)
X_subdomain_tfidf_shared = np.ndarray(
    X_subdomain_tfidf.shape, dtype=X_subdomain_tfidf.dtype, buffer=X_subdomain_shm.buf
)

np.copyto(X_domain_tfidf_shared, X_domain_tfidf)
np.copyto(X_subdomain_tfidf_shared, X_subdomain_tfidf)

# Calculate the explained variance ratio for different numbers of components
max_n_components = 2000  # Adjust based on your needs


def lsa_variance_ratio(args):
    shm_name, shape, dtype, n_components = args
    shm = shared_memory.SharedMemory(name=shm_name)
    X_tfidf = np.ndarray(shape, dtype=dtype, buffer=shm.buf)

    start = time.time()
    lsa = make_pipeline(TruncatedSVD(n_components=n_components), Normalizer(copy=False))
    lsa.fit_transform(X_tfidf)
    explained_variance_ratio = lsa[0].explained_variance_ratio_.sum()
    end = time.time()

    # Close the shared memory object in the current process
    shm.close()

    return (n_components, explained_variance_ratio, end - start)


# Parallelize the for loop using multiprocessing.Pool
with Pool() as pool:
    tasks_domain = [
        (
            X_domain_shm.name,
            X_domain_tfidf.shape,
            X_domain_tfidf.dtype,
            n_components,
            "domain",
        )
        for n_components in range(start_n_components_domain, max_n_components_domain + 1, step_domain)
    ]
    tasks_subdomain = [
        (
            X_subdomain_shm.name,
            X_subdomain_tfidf.shape,
            X_subdomain_tfidf.dtype,
            n_components,
            "subdomain",
        )
        for n_components in range(start_n_components_subdomain, max_n_components_subdomain + 1, step_subdomain)
    ]

    results_domain = pool.map(lsa_variance_ratio, tasks_domain)
    results_subdomain = pool.map(lsa_variance_ratio, tasks_subdomain)

times_explained_variance_domain = np.array(results_domain)
times_explained_variance_subdomain = np.array(results_subdomain)

# Clean up shared memory blocks
X_domain_shm.close()
X_domain_shm.unlink()
X_subdomain_shm.close()
X_subdomain_shm.unlink()

In [ ]:
from multiprocessing import Pool, shared_memory

# Create shared memory blocks for the arrays
X_domain_shm = shared_memory.SharedMemory(create=True, size=X_domain_tfidf.nbytes)
X_subdomain_shm = shared_memory.SharedMemory(create=True, size=X_subdomain_tfidf.nbytes)

# Copy the arrays to shared memory
X_domain_tfidf_shared = np.ndarray(
    X_domain_tfidf.shape, dtype=X_domain_tfidf.dtype, buffer=X_domain_shm.buf
)
X_subdomain_tfidf_shared = np.ndarray(
    X_subdomain_tfidf.shape, dtype=X_subdomain_tfidf.dtype, buffer=X_subdomain_shm.buf
)

np.copyto(X_domain_tfidf_shared, X_domain_tfidf)
np.copyto(X_subdomain_tfidf_shared, X_subdomain_tfidf)

# Calculate the explained variance ratio for different numbers of components
max_n_components_domain = 2000  # Adjust based on your needs
max_n_components_subdomain = 200  # Adjust based on your needs
start_n_components_domain = 500
start_n_components_subdomain = 10
step_domain = 100
step_subdomain = 10


def lsa_variance_ratio(args):
    X_tfidf_shm, n_components = args
    X_tfidf = np.ndarray(
        X_tfidf_shm.shape, dtype=X_tfidf_shm.dtype, buffer=X_tfidf_shm.buf
    )

    start = time.time()
    lsa = make_pipeline(TruncatedSVD(n_components=n_components), Normalizer(copy=False))
    lsa.fit_transform(X_tfidf)
    explained_variance_ratio = lsa[0].explained_variance_ratio_.sum()
    end = time.time()
    return (n_components, explained_variance_ratio, end - start)


# Parallelize the for loop using multiprocessing.Pool
with Pool() as pool:
    tasks_domain = [
        (X_domain_tfidf_shared, n_components, "domain")
        for n_components in range(
            start_n_components_domain, max_n_components_domain + 1, step_domain
        )
    ]
    tasks_subdomain = [
        (X_subdomain_tfidf_shared, n_components, "subdomain")
        for n_components in range(
            start_n_components_subdomain, max_n_components_subdomain + 1, step_subdomain
        )
    ]

    results_domain = pool.map(lsa_variance_ratio, tasks_domain)
    results_subdomain = pool.map(lsa_variance_ratio, tasks_subdomain)

times_explained_variance_domain = np.array(results_domain)
times_explained_variance_subdomain = np.array(results_subdomain)

# Clean up shared memory blocks
X_domain_shm.close()
X_domain_shm.unlink()
X_subdomain_shm.close()
X_subdomain_shm.unlink()

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# Calculate the explained variance ratio for different numbers of components
max_n_components_domain = 2000  # Adjust based on your needs
max_n_components_subdomain = 200  # Adjust based on your needs
start_n_components_domain = 500
start_n_components_subdomain = 10
step_domain = 100
step_subdomain = 10


def lsa_variance_ratio(X_tfidf, n_components, feature):
    start = time.time()
    lsa = make_pipeline(TruncatedSVD(n_components=n_components), Normalizer(copy=False))
    lsa.fit_transform(X_tfidf)
    explained_variance_ratio = lsa[0].explained_variance_ratio_.sum()
    end = time.time()
    return (n_components, explained_variance_ratio, end - start, feature)


# Parallelize the for loop using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    futures_domain = [
        executor.submit(lsa_variance_ratio, X_domain_tfidf, n_components, "domain")
        for n_components in range(
            start_n_components_domain, max_n_components_domain + 1, step_domain
        )
    ]
    futures_subdomain = [
        executor.submit(
            lsa_variance_ratio, X_subdomain_tfidf, n_components, "subdomain"
        )
        for n_components in range(
            start_n_components_subdomain, max_n_components_subdomain + 1, step_subdomain
        )
    ]

times_explained_variance_domain = np.array(
    [future.result() for future in futures_domain]
)
times_explained_variance_subdomain = np.array(
    [future.result() for future in futures_subdomain]
)

# Open a file and use dump()
with open("domain_tev.pkl", "wb") as file:

    # A new file will be created
    pickle.dump(times_explained_variance_domain, file)


# Open a file and use dump()
with open("subdomain_tev.pkl", "wb") as file:

    # A new file will be created
    pickle.dump(times_explained_variance_subdomain, file)

### Encode labels

## Computing density of matrices & print variance

In [32]:
def preprocess_tfidf(df, columns, ngram_range, n_components):
    fitted_vectorizers = fit_vectorizers(df, columns, ngram_range)
    tfidf_matrices = get_tfidf_matrices(fitted_vectorizers)
    
    # Compute density of tfidf output
    for X_tfidf in tfidf_matrices:
        print(f"{X_tfidf.nnz / np.prod(X_tfidf.shape)}")

    # tfidf_matrices are sparse

    tfidf_lsas, explained_variances = lsa_selection(tfidf_matrices, n_components)

    # Compute density of LSA output
    for tfidf_lsa in tfidf_lsas:
        density = np.count_nonzero(tfidf_lsa) / float(tfidf_lsa.size)
        print(density)

    # tfidf_lsa are dense matrices

    for i, variance in enumerate(explained_variances):
        print(
            f"Explained variance of {columns[i]} in the SVD step: {variance * 100:.1f}%"
        )

    return tfidf_lsas